In [10]:
!conda activate Jatin_GPU_ENV

In [11]:
#Setting the encoding to prevent any conversion problem
import os
os.environ['LC_ALL'] = 'C.UTF-8'

In [27]:
#Important base installations
!pip install transformers
!pip install datasets
!pip install bitsandbytes
!pip install accelerate
!pip install huggingface_hub
!pip install wandb
!pip install peft
!pip install --user protobuf==3.20.*
!pip install -U peft
!pip install -U trl

     ---------------------------------------- 0.0/245.2 kB ? eta -:--:--
     ------------------------------------- 245.2/245.2 kB 14.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/102.0 kB ? eta -:--:--
     -------------------------------------- 102.0/102.0 kB 5.7 MB/s eta 0:00:00


In [13]:
from huggingface_hub import notebook_login

notebook_login()

In [35]:
#All the imports
import torch
import wandb
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from transformers import pipeline
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

In [31]:
#Models & Database Name
base_model = "mistralai/Mistral-7B-v0.1"
dataset_name, new_model = "gathnex/Gath_baize", "mistral-7b-tuned"

In [17]:
#Setting up the model
device = "cuda" # the device to load the model onto
##Setting up the BnB Configuration:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map={"": 0}
)
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [20]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

In [25]:
prompt = "As a data scientist, can you explain the concept of regularization in machine learning"
sequences = pipe(
    prompt,
    do_sample=True,
    max_new_tokens=100, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    num_return_sequences=1,
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [26]:
print(sequences[0]['generated_text'])

As a data scientist, can you explain the concept of regularization in machine learning?

[00:01:14] Rishabh: Yes, of course. So, I think that's a very good question. Regularization is a technique that is used in order to prevent the model from overfitting. So, what is overfitting? Overfitting is when the model starts to perform well on the training data, but as soon as it is presented with data that it hasn't seen before, the performance of the model


In [30]:
secret_wandb = "87cc979c732611f7735855565b27018397706a5c"
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine tuning mistral 7B', 
    job_type="training", 
    anonymous="allow"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
#Setting up the database to load
dataset = load_dataset(dataset_name,split="train")
#Sample datapoint
dataset[0]

{'chat_sample': 'The conversation between Human and AI assisatance named Gathnex [INST] Generate a headline given a content block.\nThe Sony Playstation 5 is the latest version of the console. It has improved graphics and faster processing power.\n[/INST] Experience Amazing Graphics and Speed with the New Sony Playstation 5',
 'dataset_origin': 'alpaca'}

In [34]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)


In [36]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [38]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="chat_sample",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Map:   0%|          | 0/210311 [00:00<?, ? examples/s]

d:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\trl\trainer\sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
d:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [39]:
trainer.train()

  0%|          | 0/52578 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
d:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.8169, 'grad_norm': 1.2787104845046997, 'learning_rate': 0.0002, 'epoch': 0.0}


d:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.0287, 'grad_norm': 3.381063938140869, 'learning_rate': 0.0002, 'epoch': 0.0}


...

{'loss': 0.8976, 'grad_norm': 2.4204397201538086, 'learning_rate': 0.0002, 'epoch': 0.02}


d:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.6926, 'grad_norm': 1.0647668838500977, 'learning_rate': 0.0002, 'epoch': 0.02}


d:\Anaconda3\envs\Jatin_GPU_ENV\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
